In [1]:
from common import *
from matrix import *
from helper.weighted import *
%matplotlib inline
canvas(16, 12)
import pickle
import gc

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/common.py:701: RuntimeWarning: divide by zero encountered in log
  return func(obj, *args, **kwargs)


In [2]:
from models.thealg import TheAlg
from models.split import split3

In [3]:
o = O()
def_W(o); W = o.Wa.astype(np.float32)
def_Y(o); Y = o.Ya.astype(np.float32)

#### x features {

In [4]:
of = O()

In [5]:
def_X(o, include='logs z1 wqtl wnrm')
of.Xwqtl = o.Xwqtl
of.Xwnrm = o.Xwnrm
of.Xlogz1s = o.Xlogz1s.astype(np.float32)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/common.py:701: RuntimeWarning: divide by zero encountered in log
  return func(obj, *args, **kwargs)
/home/ec2-user/Notebooks/g-research-ffc2018/helper/weighted.ipynb:11: RuntimeWarning: invalid value encountered in true_divide
  "import notebook_import_hook\n",
/home/ec2-user/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1901: RuntimeWarning: invalid value encountered in greater
  cond1 = (0 < q) & (q < 1)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1901: RuntimeWarning: invalid value encountered in less
  cond1 = (0 < q) & (q < 1)


In [6]:
of.bias = dh[['Count']].makesw().fillna(1).astype(np.float32)
of.bias.columns = pd.MultiIndex.from_product([['bias'], range(3023)], names=['Feature', 'Stock'])

In [7]:
of.xfd = dh[['Day']]
of.xfdw = ((of.xfd + 2) % 7).colname_append('.wkd')
of.Xfdt = pd.concat([of.xfd, of.xfdw], axis=1).pipe(makesw).pipe(o._dewmean).astype(np.float32)

In [8]:
save_dir = '/big/data/features/'
pathname = save_dir + '{}sw32/{}'
xabcs = dict(
    X3int = ('', 'x3.log.zc(-93).mintco0[4,1]'),
    Xgint = ('', 'xg.log.zc(-93).mintco0[6,1][3,2]'),
    X3intwqtl = ('', 'x3.log.zc(-93).mintco0[4,1].wqtl'),
    Xgintwqtl = ('', 'xg.log.zc(-93).mintco0[6,1][3,2].wqtl'),
    X3intwnrm = ('', 'x3.log.zc(-93).mintco0[4,1].wnrm'),
    Xgintwnrm = ('', 'xg.log.zc(-93).mintco0[6,1][3,2].wnrm'),
    Xowind = ('window/', 'xo.log.z(1).windiff'),
    X3wind = ('window/', 'x3.log.zc(-93).mintco0[2,1].windiff'),
    Xgwind = ('window/', 'xg.log.zc(-93).mintco0[2,1].windiff'),
    Xowindwqtl = ('window/', 'xo.log.z(1).windiff.wqtl'),
    X3windwqtl = ('window/', 'x3.log.zc(-93).mintco0[2,1].windiff.wqtl'),
    Xgwindwqtl = ('window/', 'xg.log.zc(-93).mintco0[2,1].windiff.wqtl'),
    Xowindwnrm = ('window/', 'xo.log.z(1).windiff.wnrm'),
    X3windwnrm = ('window/', 'x3.log.zc(-93).mintco0[2,1].windiff.wnrm'),
    Xgwindwnrm = ('window/', 'xg.log.zc(-93).mintco0[2,1].windiff.wnrm'),
)
for xname, abc in xabcs.items():
    of[xname] = pd.read_hdf(pathname.format(*abc))
xabcs['Xwnrm'] = ('', 'xo.log.z(1).wnrm')
for xname in 'X3int Xgint Xowind Xgwind X3wind'.split():
    of[xname] = of[xname].pipe(o._dewmean)

optional {

In [11]:
makepos = lambda X: X.clip(lower=0).colname_append('.pos', level=0).pipe(o._dewmean).astype(np.float32)
makeneg = lambda X: X.clip(upper=0).colname_append('.neg', level=0).pipe(o._dewmean).astype(np.float32)

In [19]:
for x in 'Xwnrm X3intwnrm Xgintwnrm Xowindwnrm X3windwnrm Xgwindwnrm'.split():
    print(x)
    xp = of[x].pipe(makepos)
    xp.to_hdf(pathname.format(*xabcs[x]) + '.pos', key='a')
    xn = of[x].pipe(makeneg)
    xn.to_hdf(pathname.format(*xabcs[x]) + '.neg', key='a')
    del of[x]

X3intwnrm
Xgintwnrm
Xowindwnrm
X3windwnrm
Xgwindwnrm


} optional end

In [14]:
xraw = 'bias Xfdt Xlogz1s X3int Xgint Xowind X3wind Xgwind'.split()
xq = 'Xwqtl X3intwqtl Xgintwqtl Xowindwqtl X3windwqtl Xgwindwqtl'.split()
xpn = 'Xwnrm X3intwnrm Xgintwnrm Xowindwnrm X3windwnrm Xgwindwnrm'.split()

In [10]:
for x in xpn:
    of[x + 'p'] = pd.read_hdf(pathname.format(*xabcs[x]) + '.pos')
    of[x + 'n'] = pd.read_hdf(pathname.format(*xabcs[x]) + '.neg')

In [15]:
X = pd.concat([of[x] for x in xraw] + [of[x] for x in xq] + [of[x + 'p'] for x in xpn] + [of[x + 'n'] for x in xpn],
              axis=1, copy=False).astype(np.float32, copy=False)

In [16]:
assert X.columns.levels[0].shape[0] * 3023 == X.columns.__len__()
assert X.columns.levels[1].shape[0] == 3023

In [4]:
temp = '4allin'
#X.to_hdf('/big/data/ready/algrun' + temp, key='X')
X = pd.read_hdf(top_dir + 'data/ready/algrun' + temp, key='X')

In [5]:
try:
    dict.clear(o)
    del o
except NameError:
    pass
gc.collect()
try:
    dict.clear(of)
    del of
except NameError:
    pass
gc.collect()

0

#### } x features end

In [6]:
sfilt = stock_nvdf(dh, 246) & (stockmarket == 3)

In [7]:
W = W.T[sfilt].T
Y = Y.T[sfilt].T
X = X.wide_filter(sfilt).chain.fillna(0, inplace=True)
gc.collect()

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


37

In [8]:
s = split3(X, repeat=2)
ta = TheAlg(X, Y, W, s)

In [10]:
try:
    del X, Y, W
except NameError:
    pass
gc.collect()

30729

In [11]:
assert ta.columns.get_level_values(0).nunique() * ta.columns.get_level_values(1).nunique() == ta.columns.shape[0]
assert ta.k * ta.m == ta.columns.shape[0]

In [ ]:
branch_at_depth = lambda d: [10, 2, 1, 5][d] if d < 4 else 1
ta.alg1(branch_at_depth=branch_at_depth, max_depth=4, dot=10, numdot=10)

0.9181820134788475 0.9305867972783588 0
[] ()

<1283> 0.........100.........200.........300.........400.........500.........600.........700.........800.........900.........1000.........1100.........1200........
0.821541480114473 0.8321170860186526 1
[x3.log.zc(-93).sgnint(01/34)] (34,)

<1282> 0.........100.........200.........300.........400.........500.........600.........700.........800.........900.........1000.........1100.........1200........
0.801516341039833 0.8016315569774114 2
[x3.log.zc(-93).sgnint(01/34), xg.log.zc(-93).sgnint(3/4).wnrm.pos] (34, 693)

<1281> 0.........100.........200.........300.........400.........500.........600.........700.........800.........900.........1000.........1100.........1200........
0.7969559200636617 0.7933717238181721 3
[x3.log.zc(-93).sgnint(01/34), x0.log.z(1).window(<20>*!-), xg.log.zc(-93).sgnint(3/4).wnrm.pos] (34, 184, 693)

<1280> 0.........100.........200.........300.........400.........500.........600..

## diagnosis analysis

In [32]:
pname = '[~246&m3]allin'
featnames = [str(ta.columns[ta.m * f][0]) for f in range(ta.k)]

In [33]:
with open(top_dir + 'data/progress/linsearch/' + pname + '.pkl', 'wb') as file:
    pickle.dump((featnames, ta.scores), file, protocol=-1)

In [27]:
see = {tuple(featnames[j] for j in k): s for k, s in ta.scores.items() if k is not None}
tc0, ho0 = ta.scores[None]
sub = [(tc / tc0, ho / ho0, k) for k, (tc, ho) in see.items()
       if k is not None if len(k) == 1]
[print(line) for line in sorted(sub)]

(0.5063897241171944, 0.5377377698072504, ('xg.log.zc(-93).sgnint(01/44)',))
(0.5066471120474862, 0.5384943053598569, ('xg.log.zc(-93).sgnint(03/45)',))
(0.5080230011654192, 0.5352609813830279, ('xg.log.zc(-93).sgnint(00/45)',))
(0.5117916531745474, 0.5392651560426919, ('xg.log.zc(-93).sgnint(0/4)',))
(0.5129856822574244, 0.5406864737304947, ('xg.log.zc(-93).sgnint(00/45).wnrm.neg',))
(0.5131517523247701, 0.5360009256882767, ('xg.log.zc(-93).sgnint(01/44).wnrm.neg',))
(0.5143530803824173, 0.5479084503239907, ('xg.log.zc(-93).sgnint(03/44)',))
(0.5143648213489563, 0.535197933383552, ('xg.log.zc(-93).sgnint(0/4).wnrm.neg',))
(0.5148909631029147, 0.5392364117428747, ('x3.log.zc(-93).sgnint(01/34)',))
(0.5149092688256909, 0.5429367168482829, ('xg.log.zc(-93).sgnint(03/45).wnrm.neg',))
(0.5153441597902821, 0.5388287190725686, ('xg.log.zc(-93).sgnint(02/44)',))
(0.5165843725244542, 0.5407115754173907, ('x3.log.zc(-93).sgnint(02/34)',))
(0.5183820273191309, 0.5436152193999618, ('xg.log.zc(-93)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,